In [12]:
%run load-packages.ipynb
%run stats-libs.ipynb

3.7.6 (tags/v3.7.6:43364a7ae0, Dec 19 2019, 00:42:30) [MSC v.1916 64 bit (AMD64)]
Matplotlib version 3.1.2
Numpy version 1.18.1
Pandas version 0.25.3
Scipy version 1.4.1
Seaborn version 0.10.0
Statsmodels version 0.11.1
Sklearn version 0.22.1

Set Pandas float printout to three digits ... 


Enabling interactive shell outputs ...
   Use command pass; to disable cell text outputs

Setting seaborn and matplotlib graphic parameters ...


Load custom statistics functions  ... 
check_anova_normality(group, x)  ...
check_normality(x)  ...
gen_factorial_matrix(num_factors, interaction=True, num_replications=1)  ...
check_linreg_residuals(model)  ...
lr_logit_test(model)  ...
pseudo_r2(model, y)  ...
gof_logit_test(model, y)  ...


# 3)
The effect of five different ingredient (A, B, C, D, E) on the reaction time of a chemical process is being studied. Each batch of new material is only large enough to permit five runs to be made. furthermore, each run requires approximately $\displaystyle 1\frac{1}{2}$ hours, so only five runs can be made in one day. The experimenter decides to run the experiment as a Latin square so that day and batch effects may be systematically controlled. She obtains the data that follow. Analyzed the data from this experiment (use $\alpha = 0.05$) and draw conclusions.

## load dataset

In [13]:
df4 = pd.read_excel('data\\HW1-Dataset.xlsx',sheet_name = 'Ingredients-Time',header = 0)
df4

,Time,Batch,Day,Ingredients
0,8,1,1,A
1,7,1,2,B
2,1,1,3,D
3,7,1,4,C
4,3,1,5,E
5,11,2,1,C
6,2,2,2,E
7,7,2,3,A
8,3,2,4,D
9,8,2,5,B


In [14]:
p = len(df4.Ingredients.unique())
N = p*p
y = df4.Time
total_y = np.sum(y)
p
N
total_y

5

25

147

## find sum of square and mean square of treatment,batch and day

In [15]:
y_ing = df4.groupby('Ingredients').aggregate({'Time':'sum'}).Time
SS = np.sum(y_ing**2)
SSTr = (1/p)*SS - (total_y**2)/N
MSTr = SSTr/(p-1)
SSTr, MSTr

(141.44000000000005, 35.360000000000014)

In [16]:
y_row = df4.groupby('Batch').aggregate({'Time':'sum'}).Time
SS = np.sum(y_row**2)
SSRow = (1/p)*SS - (total_y**2)/N
MSRow = SSRow/(p-1)
SSRow,MSRow

(15.440000000000055, 3.8600000000000136)

In [17]:
y_col = df4.groupby('Day').aggregate({'Time':'sum'}).Time
SS = np.sum(y_col**2)
SScol = (1/p)*SS - (total_y**2)/N
MScol = SScol/(p-1)
SScol,MScol

(12.240000000000009, 3.0600000000000023)

## find sum of square error and mean of square error

In [18]:
SST = np.sum(y**2) - (total_y**2)/N
SSE = SST - SSTr - SSRow - SScol
MSE = SSE/((p-2)*(p-1))
SSE,MSE

(37.51999999999987, 3.126666666666656)

## all of SS,MS and  F0,P-value

In [19]:
print("SST = {:.1f}, SSTr = {:.1f}, SSRow = {:.1f}, SSCcol = {:.1f}, SSE = {:.1f}".format(SST, SSTr, SSRow, SScol, SSE))
print("MSTr = {:.1f}, MSRow = {:.1f}, MScol = {:.1f}, MSE = {:.1f}".format(MSTr, MSRow, MScol, MSE))
F0 = MSTr/MSE
alpha=0.05
p_val = stats.f.sf(F0, p-1,(p-2)*(p-1))
crit_val = stats.f.isf(0.05, p-1, (p-2)*(p-1))
print("Critical value at {:.2f} = {:.2f}".format(alpha, crit_val))
print("F0 = {:.3f}, P-value = {:.3f}".format(F0, p_val))

SST = 206.6, SSTr = 141.4, SSRow = 15.4, SSCcol = 12.2, SSE = 37.5
MSTr = 35.4, MSRow = 3.9, MScol = 3.1, MSE = 3.1
Critical value at 0.05 = 3.26
F0 = 11.309, P-value = 0.000


The results shows that F-statistics $\displaystyle F_0 = \frac{MS_{TR}}{MS_{E}}$ for testing treatment effect by comparing with $F(p-1,(p-1)(p-2))$ is more than critical value. It can summary that there are treatment effect in this dataset due to a rejection of hypothesis $H_0: \tau_i = 0;\quad H_1: \tau_i \neq 0$.

## Tukey HSD test

In [20]:
group_mean = df4.groupby('Ingredients').aggregate({'Time':'mean'}).Time
group_mean

Ingredients
A   8.400
B   5.600
C   8.800
D   3.400
E   3.200
Name: Time, dtype: float64

In [21]:
from statsmodels.stats.libqsturng import psturng, qsturng
margin_err = qsturng(0.90, p, (p-2)*(p-1))*np.sqrt(MSE/p)
import itertools
allpairs=list(itertools.combinations(sorted(df4.Ingredients.unique()),2))
tukey_df = pd.DataFrame(allpairs, columns={'group1','group2'})
tukey_df['meandiff'] = tukey_df.apply(lambda x: group_mean[x['group2']] - group_mean[x['group1']], axis=1)
tukey_df['lower'] = tukey_df.apply(lambda x: x['meandiff'] - margin_err, axis=1)
tukey_df['upper'] = tukey_df.apply(lambda x: x['meandiff'] + margin_err, axis=1)
tukey_df['reject'] = tukey_df.apply(lambda x: 'False' if x['lower'] < 0 and x['upper'] > 0 else 'True', axis=1)
tukey_df

,group1,group2,meandiff,lower,upper,reject
0,A,B,-2.800,-5.901,0.301,False
1,A,C,0.400,-2.701,3.501,False
2,A,D,-5.000,-8.101,-1.899,True
3,A,E,-5.200,-8.301,-2.099,True
4,B,C,3.200,0.099,6.301,True
5,B,D,-2.200,-5.301,0.901,False
6,B,E,-2.400,-5.501,0.701,False
7,C,D,-5.400,-8.501,-2.299,True
8,C,E,-5.600,-8.701,-2.499,True
9,D,E,-0.200,-3.301,2.901,False


From the results as shown, there are 5 of 10 pairs that have significantly different mean. These are, A and D, A and E, B and C, C and D, C and E.

## Efficiency of blocking variables

In [22]:
E1 = (MSRow+MScol+(p-1)*MSE)/((p+1)*MSE)
E2 = (MScol+(p-1)*MSE)/(p*MSE)
E3 = (MSRow+(p-1)*MSE)/(p*MSE)
print('E1 = {:.2f}, E2 = {:.2f}, E3 = {:.2f}'.format(E1,E2,E3))

E1 = 1.04, E2 = 1.00, E3 = 1.05


The blocking efficiency in these blockings are relatively equal.